# Security — Report

This notebook visualizes the CSV outputs generated by the **Security** block.

## What this notebook shows
- **Security configurations**: Deprecated adapter usage (Donut), parent→class overview (Sunburst), annotation density (Treemap).
- **Spring Security (annotations)**: Global annotation popularity (Donut), class→annotation breakdown (Sunburst), top classes by annotated methods (Treemap).
- **Potentially unsecured endpoints**: HTTP method distribution (Donut), controller→method view (Sunburst), controllers with most endpoints (Treemap), compact top list.

> If a CSV is missing or empty, the cell prints an info message and skips the chart.


In [ ]:
# Setup & helpers
# - CSVs are read from reports/csv-reports/<CATEGORY>/<file>.csv relative to this notebook folder.
# - Minimal console output; only show information if a CSV is missing/empty.
# - Titles are standardized without block prefixes.
# - If you add bar charts later, set `color_discrete_sequence=DEFAULT_BAR_COLOR`.

import os
from pathlib import Path
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display

pd.set_option('future.no_silent_downcasting', True)

CATEGORY = "Security"
CSV_BASE = Path("../reports/csv-reports").resolve()
SEC_DIR = CSV_BASE / CATEGORY

# Explicit default color for future bar charts (not used by pies/treemaps/sunbursts here)
DEFAULT_BAR_COLOR = ["#1f77b4"]

# CSV IO helpers
NA_LITS = ["", " ", "NA", "N/A", "n/a", "NaN", "NULL", "Null", "null", "None", "none", "-", "--"]

def read_csv_safe(p: Path) -> pd.DataFrame:
    """Read a CSV if present; otherwise return an empty DataFrame.
    Prints a minimal info message when missing or unreadable."""
    p = Path(p)
    if not p.exists():
        print(f"[info] Missing CSV: {p}")
        return pd.DataFrame()
    try:
        df = pd.read_csv(p, na_values=NA_LITS, keep_default_na=True)
        df.columns = [str(c).strip() for c in df.columns]
        return df.dropna(how="all")
    except Exception as e:
        print(f"[warn] Failed to read {p}: {e}")
        return pd.DataFrame()

def find_col(df, *cands, default=None, contains=None):
    """Return a column name by exact candidate(s) or substring (case-insensitive)."""
    if df is None or df.empty:
        return default
    low = {c.lower(): c for c in df.columns}
    for c in cands:
        if c and c.lower() in low:
            return low[c.lower()]
    if contains:
        for k, orig in low.items():
            if contains.lower() in k:
                return orig
    return default

MAX_ROWS_PREVIEW = 5
MAX_BARS = 30

# Import shared chart functions from interface
import sys
sys.path.append(str(Path("../interface").resolve()))
from charts.security_charts import (
    create_deprecated_adapter_donut,
    create_parent_class_sunburst,
    create_annotation_density_treemap,
    create_config_methods_pie,
    create_annotations_popularity_donut,
    create_class_annotation_sunburst,
    create_top_classes_treemap,
    create_http_method_donut,
    create_controller_method_sunburst,
    create_controllers_treemap
)


## 1) Security configurations

In [ ]:
# Charts generated here:
#  - 1A) Uses deprecated adapter? (Donut)
#  - 1B) Parent class → config class (Sunburst; size = configMethodsCount)
#  - 1C) Annotation density per config class (Treemap)

path = SEC_DIR / "Security_Configurations.csv"
df_sc = read_csv_safe(path)

c_cls   = find_col(df_sc, "securityConfigClass", contains="config")
c_ext   = find_col(df_sc, "extendsClass", contains="extend", default="extendsClass")
c_annc  = find_col(df_sc, "annotationsCount", contains="annot", default="annotationsCount")
c_cfgc  = find_col(df_sc, "configMethodsCount", contains="config", default="configMethodsCount")
c_cfgs  = find_col(df_sc, "configMethods", contains="config", default="configMethods")
c_depr  = find_col(df_sc, "usesDeprecatedAdapter", contains="deprecated", default="usesDeprecatedAdapter")

required = [c_cls, c_ext, c_annc, c_cfgc, c_depr]
if df_sc.empty or any(col is None for col in required):
    print("[info] No data for Security_Configurations (missing CSV or required columns).")
else:
    # Small preview
    display(df_sc[[c_cls, c_ext, c_annc, c_cfgc, c_depr]].head(MAX_ROWS_PREVIEW))

    # Ensure numeric/bool types
    df_sc[c_annc] = pd.to_numeric(df_sc[c_annc], errors="coerce").fillna(0)
    df_sc[c_cfgc] = pd.to_numeric(df_sc[c_cfgc], errors="coerce").fillna(0)
    if df_sc[c_depr].dtype != bool:
        df_sc[c_depr] = df_sc[c_depr].astype(str).str.lower().isin(["true","1","yes","y"])

    # 1A) Deprecated adapter usage (Donut)
    fig = create_deprecated_adapter_donut(df_sc, c_depr)
    if fig:
        fig.show()

    # 1B) Parent → class (Sunburst; size = configMethodsCount)
    fig = create_parent_class_sunburst(df_sc, c_ext, c_cls, c_cfgc)
    if fig:
        fig.show()

    # 1C) Treemap by annotation count per class
    fig = create_annotation_density_treemap(df_sc, c_cls, c_annc)
    if fig:
        fig.show()

    # Optional exploded view of config methods if present
    if c_cfgs:
        fig = create_config_methods_pie(df_sc, c_cls, c_cfgs)
        if fig:
            fig.show()


## 2) Spring Security — annotated methods

In [ ]:
# Charts generated here:
#  - 2A) Security annotations used (global) (Donut)
#  - 2B) Annotated methods by class and annotation (Sunburst)
#  - 2C) Top classes by # of security-annotated methods (Treemap)

path = SEC_DIR / "Spring_Security.csv"
df_ss = read_csv_safe(path)

c_decl = find_col(df_ss, "declaringClass", contains="declaring")
c_meth = find_col(df_ss, "methodName", contains="method")
c_ann  = find_col(df_ss, "annotationName", contains="annot")

required = [c_decl, c_meth, c_ann]
if df_ss.empty or any(col is None for col in required):
    print("[info] No data for Spring_Security (missing CSV or required columns).")
else:
    display(df_ss[[c_decl, c_meth, c_ann]].head(MAX_ROWS_PREVIEW))

    # 2A) Annotations popularity (Donut)
    fig = create_annotations_popularity_donut(df_ss, c_ann)
    if fig:
        fig.show()

    # 2B) Sunburst DeclaringClass → Annotation (size = count)
    fig = create_class_annotation_sunburst(df_ss, c_decl, c_ann)
    if fig:
        fig.show()

    # 2C) Treemap of classes by number of annotated methods (Top N)
    fig = create_top_classes_treemap(df_ss, c_decl)
    if fig:
        fig.show()


## 3) Potentially unsecured endpoints

In [ ]:
# Charts generated here:
#  - 3A) Potentially unsecured endpoints by HTTP method (Donut)
#  - 3B) Potentially unsecured endpoints by controller and method (Sunburst)
#  - 3C) Controllers with most potentially unsecured endpoints (Treemap)
#  - 3D) Compact top endpoints list (table)

path = SEC_DIR / "Unsecured_Endpoints.csv"
df_ue = read_csv_safe(path)

c_ctrl = find_col(df_ue, "Controller", contains="controller")
c_meth = find_col(df_ue, "Method", contains="method")
c_http = find_col(df_ue, "HttpMethod", contains="http")
c_ep   = find_col(df_ue, "CompleteEndpoint", contains="endpoint")
c_stat = find_col(df_ue, "SecurityStatus", contains="security", default="SecurityStatus")

required = [c_ctrl, c_meth, c_http, c_ep, c_stat]
if df_ue.empty or any(col is None for col in required):
    print("[info] No data for Unsecured_Endpoints (missing CSV or required columns).")
else:
    display(df_ue[[c_ctrl, c_http, c_ep]].head(MAX_ROWS_PREVIEW))

    # 3A) Distribution by HTTP method (Donut)
    fig = create_http_method_donut(df_ue, c_http)
    if fig:
        fig.show()

    # 3B) Sunburst Controller → HTTP Method (size = count)
    fig = create_controller_method_sunburst(df_ue, c_ctrl, c_http)
    if fig:
        fig.show()

    # 3C) Treemap by Controller (Top N)
    fig = create_controllers_treemap(df_ue, c_ctrl)
    if fig:
        fig.show()

    # 3D) Compact list of top endpoints (by simple frequency of path)
    if c_ep:
        by_ep = df_ue[c_ep].value_counts().rename_axis("endpoint").reset_index(name="count")
        display(by_ep.head(10))
    else:
        print("[info] Endpoint column not available for top list.")
